In [1]:
import numpy as np
import pandas as pd
import drawings as dr

C:\Users\RGHS\anaconda3\lib\site-packages\drawSvg\raster.py:18: RuntimeWarning: Failed to import CairoSVG. drawSvg will be unable to output PNG or other raster image formats. See https://github.com/cduck/drawSvg#prerequisites for more details.
Original OSError: no library called "cairo" was found
cannot load library 'C:\GTK\bin\libcairo-2.dll': error 0xc1
cannot load library 'libcairo.so.2': error 0x7e
cannot load library 'libcairo.2.dylib': error 0x7e
cannot load library 'libcairo-2.dll': error 0xc1
  warnings.warn(msg, RuntimeWarning)


In [1]:
filepath = 'testdata.csv'

In [ ]:
vscale = 500
ticks = 10

origin = 40
padding = 5
column_spacing = 40
line_weight = 0.5
manual_column_height = None
cols = None

In [ ]:
src = pd.read_csv(filepath)
src = src.fillna('NaN')
elevations = dr.elevs(src.thickness)

In [2]:
default = True

In [3]:
if(default is True):
    gs_codes, gs_widths = dr.grainsize()
    fa_codes, fa_colors = dr.faciesList()
    canv = dr.canvas()
else:
    